# Data Loading, Storage, and File Formats

In [1]:
import numpy as np
import pandas as pd

In [2]:
!cat ./examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [3]:
df = pd.read_csv("./examples/ex1.csv")
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
# a file will not always have a header row
!cat ./examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

To read a csv file with no header row into a DataFrame, you can specify the column names yourself or let pandas assign default column names.

In [5]:
pd.read_csv("./examples/ex2.csv", header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
pd.read_csv("./examples/ex2.csv", names=["a", "b", "c", "d", "message"])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
# you can specify the message column to be the index of the returned DataFrame
names = ["a", "b", "c", "d", "message"]
pd.read_csv("./examples/ex2.csv", names=names, index_col="message")

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [8]:
# you can also specify multiple columns to be used as the index
!cat ./examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [9]:
parsed = pd.read_csv("./examples/csv_mindex.csv", index_col=["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases a table might not have a fixed delimiter. While you could do some munging by hand, the csv module allows you to define a regular expression to handle the delimiters.

In [10]:
!cat examples/ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


In [11]:
# \s+ means one or more whitespace characters
result = pd.read_table("./examples/ex3.txt", sep="\s+")
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Because there was one fewer column name than the number of data rows, read_csv infers that the first column should be the DataFrame’s index in this special case.

In [12]:
!cat examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [13]:
# skip the first, third, and fourth rows of a file with skiprows
pd.read_csv("./examples/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Handling missing values is an important and frequently nuanced part of the file parsing process. Missing data is usually either not present (empty string) or marked by some sentinel value. By default, pandas uses a set of commonly occurring sentinels, such as NA and NULL:

In [14]:
!cat examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo

In [15]:
result = pd.read_csv("./examples/ex5.csv")
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [16]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


The na_values option accepts a sequence of strings to add to the default list of strings recognized as NA values.

In [17]:
result = pd.read_csv("./examples/ex5.csv", na_values=["NULL"])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


When processing very large files or figuring out the right set of arguments to correctly process a large file, you may want to read only a small piece of a dile or iterate through smaller chunks of the file.

In [18]:
# make the pandas display more compact
pd.options.display.max_rows = 10

In [19]:
result = pd.read_csv("./examples/ex6.csv")
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [20]:
# read just 5 rows
pd.read_csv("./examples/ex6.csv", nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [21]:
# read a file in pieces
chunker = pd.read_csv("./examples/ex6.csv", chunksize=1000)
type(chunker)

pandas.io.parsers.readers.TextFileReader

This TextFileReader object returned by pandas allows you to iterate over parth of the file according to the chunksize.

In [22]:
tot = pd.Series([], dtype="int64")
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

It's possible to load most forms of tabular data from disk using functions like pandas.read_csv. In some cases, however, some manual processing may be necessary. It's not uncommon to receive a file with one or more malformed lines that trip up read_csv. To illustrate the basic tools, consider a small CSV file.

In [23]:
!cat examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [24]:
import csv

with open("./examples/ex7.csv") as f:
    lines = list(csv.reader(f))

header, values = lines[0], lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

JSON is nearly valid Python code with the exception of its null value and some other nuances. The basic types are objects, arrays, strings, numbers, Booleans and nulls. All of the keys in an object must be strings. These are the basic types for the Python standard library json module to decode into Python objects.

In [25]:
import json

obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""

result = json.loads(obj)
result

{'name': 'Wes',
 'cities_lived': ['Akron', 'Nashville', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

In [26]:
# conver a Python object back to JSON
asjson = json.dumps(result)
asjson

'{"name": "Wes", "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"], "pet": null, "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]}, {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]}'

How you convert a JSON object or list of objects to a DataFrame or some other data structure will be up to you. Conveniently, you can pass a list of dictionaries to the DataFrame constructor and select a subset of the data fields.

In [27]:
siblings = pd.DataFrame(result["siblings"], columns=["name", "age"])
siblings

,name,age
0,Scott,34
1,Katie,42


In [28]:
# pandas.read_json can automatically convert JSON datasets in specific arrangements into a Series or DataFrame
!cat examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [29]:
data = pd.read_json("./examples/example.json")
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [30]:
# use to_json to convert a Series or DataFrame to JSON
import sys

data.to_json(sys.stdout)

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}

While Python has many libraries for reading and writing data in the ubiquitous HTML and XML formats, pandas has a built-in function, pandas.read_html, that uses libraries like lxml and Beautiful Soup to automatically parse tables out of HTML files as DataFrame objects.

In [31]:
!pip install lxml

  Using cached lxml-4.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (7.1 MB)


The pandas.read_html function has a number of options, but by default it searches for and attempts to parse all tabular data contained within \<table\> tags.

In [32]:
tables = pd.read_html("./examples/fdic_failed_bank_list.html")
len(tables)

1

In [33]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


Now we can proceed to do some data cleaning and analysis, like computing the number of bank failures by year.

In [34]:
close_timestamps = pd.to_datetime(failures["Closing Date"])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

The New York Metropolitan Transportation Authority (MTA) makes available a number of data series about its bus and train services in XML format. Here we'll look at the performance data, which is contained in a set of XML files. Each train or bus service has a different file containing monthly data as a series of XML records.

In [35]:
from lxml import objectify

path = "datasets/mta_perf/Performance_MNR.xml"

with open(path) as f:
    parsed = objectify.parse(f)

root = parsed.getroot()
root.INDICATOR

<Element INDICATOR at 0x7f2dfedefe00>

root.INDICATOR returns a generator yielding each \<INDICATOR\> XML element. For each record, we can populate a dictionary of tag names to data values.

In [36]:
data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ", "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}

    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue

        el_data[child.tag] = child.pyval

    data.append(el_data)

In [37]:
# convert this list of dictionaries into a DataFrame
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


The previous process can be reduced into a single one-line expression by using pandas' pandas.read_xml.

In [38]:
perf2 = pd.read_xml(path)
perf2.head()

,INDICATOR_SEQ,PARENT_SEQ,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,DESIRED_CHANGE,INDICATOR_UNIT,DECIMAL_PLACES,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,U,%,1,95.00,96.90,95.00,96.90
1,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,U,%,1,95.00,96.00,95.00,95.00
2,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,U,%,1,95.00,96.30,95.00,96.90
3,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,U,%,1,95.00,96.80,95.00,98.30
4,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,U,%,1,95.00,96.60,95.00,95.80


A simple way to store or serialize data in binary format is using Python's built-in pickle module. pandas objects all have a to_pickle method that writes the data to disk in pickle format.

In [39]:
frame = pd.read_csv("./examples/ex1.csv")
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [40]:
frame.to_pickle("./examples/frame_pickle")

Pickle files are in general readable only in Python. You can read any "pickled" object stores in a file by using the built-in pickle directly, or even more conveniently using pandas.read_pickle.

In [41]:
pd.read_pickle("./examples/frame_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Its important to keep in mind that using the pickle format is not recommended since is hard to guarantee that the format will be stable over time.

Pandas also supports reading tabular data stored in Excel 2003 (and higher) files using either the ExcelFile class or pandas.read_excel function.

In [42]:
!pip install openpyxl xlrd

  Using cached openpyxl-3.1.1-py2.py3-none-any.whl (249 kB)
  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [43]:
xlsx = pd.ExcelFile("examples/ex1.xlsx")
xlsx.sheet_names

['Sheet1']

In [44]:
# read the first sheet of the Excel file
xlsx.parse("Sheet1")

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [45]:
xlsx.parse(sheet_name="Sheet1", index_col=0)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [46]:
# read multiple sheets of an Excel file
frame = pd.read_excel("examples/ex1.xlsx", "Sheet1")
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


In [47]:
# write data to Excel format
writer = pd.ExcelWriter("examples/ex2.xlsx")
frame.to_excel(writer, "Sheet1")
writer.save()

/tmp/ipykernel_13735/2845250975.py:4: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


HDF5 is a respected file format intended for storing large quantities of scientific array data. The "HDF" stands for hierarchical data format. Each HDF5 file can store multiple datasets and supporting metadata. Compared with simpler formats, HDF5 supports on-the-fly compression and allows data to be appended to an existing file.

In [48]:
!pip install tables

  Using cached tables-3.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.5 MB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached blosc2-2.0.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.9 MB)
  Using cached numexpr-2.8.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (381 kB)
  Using cached Cython-0.29.33-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Using cached msgpack-1.0.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (316 kB)


In [49]:
frame = pd.DataFrame({"a": np.random.randn(100)})

store = pd.HDFStore("examples/mydata.h5")
store["obj1"] = frame
store["obj1_col"] = frame["a"]

store

<class 'pandas.io.pytables.HDFStore'>
File path: examples/mydata.h5

In [50]:
store["obj1"]

,a
0,-1.008030
1,-1.246724
2,-0.098112
3,1.283224
4,2.101730
...,...
95,-0.114484
96,-0.395579
97,-1.029374
98,-0.520599


HDFStore supports two storage schemas:
- fixed (default).
- table.

The latter is generally slower but offers more flexible query operations like selecting subsets of the data.

In [51]:
store.put("obj2", frame, format="table")
store.select("obj2", where=["index >= 10 and index <= 15"])

,a
10,0.569278
11,0.675181
12,1.299689
13,-0.511161
14,-1.338679
15,-0.022894


In [52]:
store.close()

In [53]:
frame.to_hdf("examples/mydata.h5", "obj3", format="table")
pd.read_hdf("examples/mydata.h5", "obj3", where=["index < 5"])

,a
0,-1.008030
1,-1.246724
2,-0.098112
3,1.283224
4,2.101730


If you work with large quantities of data locally, Its encouraged to explore PyTables and h5py to see how they can suit your needs since many data analysis problems are I/O bound rather than CPU bound.

In [54]:
# install requests
!pip install requests

  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Using cached charset_normalizer-3.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (198 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached urllib3-1.26.14-py2.py3-none-any.whl (140 kB)


In [55]:
import requests

url = "https://api.github.com/repos/pandas-dev/pandas/issues"

resp = requests.get(url)
resp.raise_for_status()  # check for errors
resp

<Response [200]>

In [56]:
data = resp.json()
data[0]["title"]

'Added changes for type checking'

In [57]:
issues = pd.DataFrame(data, columns=["number", "title", "labels", "state"])
issues.head()

,number,title,labels,state
0,51754,Added changes for type checking,[],open
1,51753,Add test to check numeric precision GH33234,[],open
2,51752,BUG: converting a `string[pyarrow]` column to ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
3,51751,API: Should groupby.rolling be treated as a tr...,"[{'id': 233160, 'node_id': 'MDU6TGFiZWwyMzMxNj...",open
4,51750,PERF: groupby.value_counts,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open
